In [157]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'c:\\Users\\Joaquín Amat\\Documents\\GitHub\\skforecast'

In [158]:
from skforecast.ForecasterBaseline import ForecasterLastEquivalentDate
from skforecast.model_selection import backtesting_forecaster
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
from skforecast import __version__
print(__version__)


0.10.1


In [159]:
# simulate a time series
n = 365
time = pd.date_range(start='2000-01-01', periods=n, freq='d')
y = np.arange(n)
df = pd.DataFrame({'time': time, 'y': y})
df = df.set_index('time')
df = df.asfreq('d')
df

,y
time,
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
...,...
2000-12-26,360
2000-12-27,361
2000-12-28,362


In [160]:
forecaster = ForecasterLastEquivalentDate(offset=10, n_equivalent_dates=1)
forecaster.fit(y=df['y'])
forecaster

ForecasterLastEquivalentDate 
Offset: 10 
Number of equivalent dates: 1 
Aggregation function: mean 
Window size: 10 
Training range: [Timestamp('2000-01-01 00:00:00'), Timestamp('2000-12-30 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: D 
Creation date: 2023-10-27 20:00:23 
Last fit date: 2023-10-27 20:00:23 
Skforecast version: 0.10.1 
Python version: 3.11.4 
Forecaster id: None 

In [161]:
forecaster.last_window

time
2000-12-21    355
2000-12-22    356
2000-12-23    357
2000-12-24    358
2000-12-25    359
2000-12-26    360
2000-12-27    361
2000-12-28    362
2000-12-29    363
2000-12-30    364
Freq: D, Name: y, dtype: int32

In [162]:
forecaster.predict(steps=15)

2000-12-31    355
2001-01-01    356
2001-01-02    357
2001-01-03    358
2001-01-04    359
2001-01-05    360
2001-01-06    361
2001-01-07    362
2001-01-08    363
2001-01-09    364
2001-01-10    355
2001-01-11    356
2001-01-12    357
2001-01-13    358
2001-01-14    359
Freq: D, Name: pred, dtype: int32

In [163]:
metric, predictions = backtesting_forecaster(
    forecaster=forecaster,
    y=df['y'],
    initial_train_size=int(len(df) * 0.5),
    steps=9,
    metric='mean_absolute_error',
    verbose=False
)

  0%|          | 0/21 [00:00<?, ?it/s]

In [164]:
forecaster = ForecasterLastEquivalentDate(
    offset=DateOffset(days=10),
    n_equivalent_dates=2
)
forecaster.fit(y=df['y'])
forecaster

ForecasterLastEquivalentDate 
Offset: <DateOffset: days=10> 
Number of equivalent dates: 2 
Aggregation function: mean 
Window size: 20 
Training range: [Timestamp('2000-01-01 00:00:00'), Timestamp('2000-12-30 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: D 
Creation date: 2023-10-27 20:00:27 
Last fit date: 2023-10-27 20:00:27 
Skforecast version: 0.10.1 
Python version: 3.11.4 
Forecaster id: None 

In [165]:
forecaster.last_window

time
2000-12-11    345
2000-12-12    346
2000-12-13    347
2000-12-14    348
2000-12-15    349
2000-12-16    350
2000-12-17    351
2000-12-18    352
2000-12-19    353
2000-12-20    354
2000-12-21    355
2000-12-22    356
2000-12-23    357
2000-12-24    358
2000-12-25    359
2000-12-26    360
2000-12-27    361
2000-12-28    362
2000-12-29    363
2000-12-30    364
Freq: D, Name: y, dtype: int32

In [166]:
forecaster.predict(steps=20)

2000-12-31    350.0
2001-01-01    351.0
2001-01-02    352.0
2001-01-03    353.0
2001-01-04    354.0
2001-01-05    355.0
2001-01-06    356.0
2001-01-07    357.0
2001-01-08    358.0
2001-01-09    359.0
2001-01-10    350.0
2001-01-11    351.0
2001-01-12    352.0
2001-01-13    353.0
2001-01-14    354.0
2001-01-15    355.0
2001-01-16    356.0
2001-01-17    357.0
2001-01-18    358.0
2001-01-19    359.0
Freq: D, Name: pred, dtype: float64

In [168]:
ametric, predictions = backtesting_forecaster(
    forecaster=forecaster,
    y=df['y'],
    initial_train_size=int(len(df) * 0.5),
    steps=9,
    metric='mean_absolute_error',
    verbose=True,
    n_jobs=1
)

Information of backtesting process
----------------------------------
Number of observations used for initial training: 182
Number of observations used for backtesting: 183
    Number of folds: 21
    Number of steps per fold: 9
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 3 observations.

Fold: 0
    Training:   2000-01-01 00:00:00 -- 2000-06-30 00:00:00  (n=182)
    Validation: 2000-07-01 00:00:00 -- 2000-07-09 00:00:00  (n=9)
Fold: 1
    Training:   2000-01-01 00:00:00 -- 2000-06-30 00:00:00  (n=182)
    Validation: 2000-07-10 00:00:00 -- 2000-07-18 00:00:00  (n=9)
Fold: 2
    Training:   2000-01-01 00:00:00 -- 2000-06-30 00:00:00  (n=182)
    Validation: 2000-07-19 00:00:00 -- 2000-07-27 00:00:00  (n=9)
Fold: 3
    Training:   2000-01-01 00:00:00 -- 2000-06-30 00:00:00  (n=182)
    Validation: 2000-07-28 00:00:00 -- 2000-08-05 00:00:00  (n=9)
Fold: 4
    Training:   2000-01-01 00:00:00 -- 2000-06-30 00:00:00  (n=182)

  0%|          | 0/21 [00:00<?, ?it/s]

OutOfBoundsDatetime: Cannot generate range with start=961545600000000000 and periods=-1

In [ ]:
# simulate time series with dayly frequency
n = 365
time = pd.date_range(start='2000-01-01', periods=n, freq='D')
y = np.arange(n)
y = pd.Series(y, index=time)
offset = offsets.Day(n=10)
y

NameError: name 'offsets' is not defined

In [ ]:
# Substract 10 days to the las value of the time series index and add one unit of the time series frequency
y.index[-1] - pd.DateOffset(days=10) 

Timestamp('2000-12-20 00:00:00')

In [ ]:
y.index[-1] - pd.DateOffset(days=10) + 2*y.index.freq

Timestamp('2000-12-22 00:00:00')

In [ ]:
from skforecast.utils import expand_index

last_window = y.tail(20)
last_window_index = y.index
last_window_values = y.to_numpy()
n_equivalent_dates = 2
print(last_window_index[-1])
steps = 15
index_freq = y.index.freq


predictions_index = expand_index(index=last_window_index,steps=steps)
equivalent_dates = [
                predictions_index[0] - offset + n * index_freq
                for n
                in np.arange(steps)[:len(last_window)]
            ]
equivalent_dates = np.tile(
                        equivalent_dates,
                        int(np.ceil(steps/len(equivalent_dates)))
                    )
equivalent_dates = equivalent_dates[:steps]

#substract off set element whise
equivalent_dates = [
    [
        date - offset * n
        for n
        in np.arange(n_equivalent_dates)
    ]
    for date
    in equivalent_dates
]
print(equivalent_dates)

equivalent_values = [
    last_window.loc[dates].to_numpy()
    for dates
    in equivalent_dates
]
equivalent_values


2000-12-30 00:00:00
[[Timestamp('2000-12-21 00:00:00'), Timestamp('2000-12-11 00:00:00')], [Timestamp('2000-12-22 00:00:00'), Timestamp('2000-12-12 00:00:00')], [Timestamp('2000-12-23 00:00:00'), Timestamp('2000-12-13 00:00:00')], [Timestamp('2000-12-24 00:00:00'), Timestamp('2000-12-14 00:00:00')], [Timestamp('2000-12-25 00:00:00'), Timestamp('2000-12-15 00:00:00')], [Timestamp('2000-12-26 00:00:00'), Timestamp('2000-12-16 00:00:00')], [Timestamp('2000-12-27 00:00:00'), Timestamp('2000-12-17 00:00:00')], [Timestamp('2000-12-28 00:00:00'), Timestamp('2000-12-18 00:00:00')], [Timestamp('2000-12-29 00:00:00'), Timestamp('2000-12-19 00:00:00')], [Timestamp('2000-12-30 00:00:00'), Timestamp('2000-12-20 00:00:00')], [Timestamp('2000-12-31 00:00:00'), Timestamp('2000-12-21 00:00:00')], [Timestamp('2001-01-01 00:00:00'), Timestamp('2000-12-22 00:00:00')], [Timestamp('2001-01-02 00:00:00'), Timestamp('2000-12-23 00:00:00')], [Timestamp('2001-01-03 00:00:00'), Timestamp('2000-12-24 00:00:00')],

KeyError: "[Timestamp('2000-12-31 00:00:00')] not in index"

In [ ]:

predictions = np.apply_along_axis(
    func1d=np.mean,
    axis=1,
    arr=equivalent_values
)

predictions

array([350., 351., 352., 353., 354., 355., 356., 357., 358., 359., 350.,
       351., 352., 353., 354.])

In [ ]:
equivalent_values = [
    
]

KeyError: "None of [Index([(2000-12-21 00:00:00, 2000-12-11 00:00:00),\n       (2000-12-22 00:00:00, 2000-12-12 00:00:00),\n       (2000-12-23 00:00:00, 2000-12-13 00:00:00),\n       (2000-12-24 00:00:00, 2000-12-14 00:00:00),\n       (2000-12-25 00:00:00, 2000-12-15 00:00:00),\n       (2000-12-26 00:00:00, 2000-12-16 00:00:00),\n       (2000-12-27 00:00:00, 2000-12-17 00:00:00),\n       (2000-12-28 00:00:00, 2000-12-18 00:00:00),\n       (2000-12-29 00:00:00, 2000-12-19 00:00:00),\n       (2000-12-30 00:00:00, 2000-12-20 00:00:00),\n       (2000-12-21 00:00:00, 2000-12-11 00:00:00),\n       (2000-12-22 00:00:00, 2000-12-12 00:00:00),\n       (2000-12-23 00:00:00, 2000-12-13 00:00:00),\n       (2000-12-24 00:00:00, 2000-12-14 00:00:00),\n       (2000-12-25 00:00:00, 2000-12-15 00:00:00)],\n      dtype='object')] are in the [index]"

In [ ]:
#substract off set element whise
[
    [
        date - offset * n
        for n
        in np.arange(n_equivalent_dates)
    ]
    for date
    in equivalent_dates
]

TypeError: unsupported operand type(s) for -: 'list' and 'pandas._libs.tslibs.offsets.Day'

In [ ]:
np.arange(steps)[:len(last_window)]

365